In [21]:
import ccxt

exchange = ccxt.binance({
    'enableRateLimit': True,
    'options': {
        'defaultType': 'delivery',  # 'delivery' -> Coin‐M Quarterly；'future' -> USDT‐M Quarterly
    },
})

markets = exchange.fetch_markets()
# 篩選出 Delivery（季度）合約
quarterly_markets = [
    m for m in markets
    if m.get('info', {}).get('contractType') == 'CURRENT_QUARTER'
]

# 打印部分示例
for m in quarterly_markets[:5]:
    print(m['symbol'], m['info']['symbol'], m['info']['contractType'])


BTC/USDT:USDT-250627 BTCUSDT_250627 CURRENT_QUARTER
ETH/USDT:USDT-250627 ETHUSDT_250627 CURRENT_QUARTER
BTC/USD:BTC-250627 BTCUSD_250627 CURRENT_QUARTER
ETH/USD:ETH-250627 ETHUSD_250627 CURRENT_QUARTER
ADA/USD:ADA-250627 ADAUSD_250627 CURRENT_QUARTER


In [1]:
import math
import ccxt
from tqdm.auto import tqdm
import pandas as pd


def fetch_quarterly_ohlcv(symbol, timeframe='1h', start_str='2024-01-01T00:00:00Z'):
    exchange = ccxt.binance({
        'enableRateLimit': True,
        'options': {'defaultType': 'delivery'},
    })

    since     = exchange.parse8601(start_str)
    limit     = 500

    ms_per_candle = exchange.parse_timeframe(timeframe) * 1000

    # 預估要跑幾輪
    now           = exchange.milliseconds()
    total_candles = (now - since) // ms_per_candle + 1
    total_iters   = math.ceil(total_candles / limit)

    # 建立進度條
    pbar = tqdm(total=total_iters, desc=f'Fetching {symbol} {timeframe}')

    all_ohlcv = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0]        # 更新起點
        pbar.update(1)              # 更新進度
        if len(ohlcv) < limit:
            break
        

    pbar.close()

    # 轉成 DataFrame
    df = pd.DataFrame(all_ohlcv, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume'
    ])
    # 1. 将 timestamp(毫秒) 解析为带时区的 UTC 时间
    df.index = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    # 2. 转换到美东时间（考虑夏令时）
    df.index = df.index.tz_convert('America/New_York')
    # 3. （可选）去掉 tz 信息，变成普通的本地时间索引
    df.index = df.index.tz_localize(None)
    # 4. 删除冗余的 timestamp 列
    df.drop(columns=['timestamp'], inplace=True)


    # 存成 CSV
    output_path = fr"C:\Users\Huang\Work place\Project_Iris\DataAnalysis\DataBase\BTCfuture_{timeframe}.csv"
    df.to_csv(output_path, index=True)
    print(f'已將 {symbol} {timeframe} 歷史數據儲存到：{output_path}')

if __name__ == '__main__':
    # 範例：拉取 BTC/USDT 季度合約 1 小時線
    data = fetch_quarterly_ohlcv('BTC/USDT:USDT-250627', '1h', '2024-01-01T00:00:00Z')


c:\Users\Huang\anaconda3\envs\v10-16\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching BTC/USDT:USDT-250627 1h:  32%|███▏      | 8/25 [00:03<00:08,  2.06it/s]


已將 BTC/USDT:USDT-250627 1h 歷史數據儲存到：C:\Users\Huang\Work place\Project_Iris\DataAnalysis\DataBase\BTCfuture_1h.csv


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# 读取数据
df_p = pd.read_csv(r'DataBase\BTCfuture_1h.csv',
                   parse_dates=['timestamp'],
                   index_col='timestamp')
df_x = pd.read_csv(r'DataBase\BTC_1h.csv',
                   parse_dates=['timestamp'],
                   index_col='timestamp')

# 重命名收盘价列
df_x = df_x.rename(columns={'close': 'xaut_close'})
df_p = df_p.rename(columns={'close': 'paxg_close'})

# 合并
df = pd.merge(df_x[['xaut_close']],
              df_p[['paxg_close']],
              left_index=True, right_index=True,
              how='inner')

# 计算指标： (paxg - xaut) * 2 / (paxg + xaut)
df['spread_ratio'] = (df['paxg_close'] - df['xaut_close']) * 2 / (df['paxg_close'] + df['xaut_close'])

df_plot = df.reset_index()

fig = px.bar(
    df_plot,
    x='timestamp',
    y='spread_ratio',
    title='PAXG & XAUT CLOSE SPREAD（(P−X)*2/(P+X)）',
    labels={
        'timestamp': 'time',
        'spread_ratio': 'SPREAD'
    }
)
fig.update_layout(
    plot_bgcolor='white',      # 绘图区背景
    paper_bgcolor='white',     # 整体画布背景
)

# # 5. 设置 x 轴刻度与滑块
# fig.update_xaxes(
#     tickformat='%H:%M',
#     dtick=60 * 1000,
#     rangeslider=dict(visible=True)
# )

fig.update_yaxes(
    tickformat='.2%'   # 两位小数的百分比；改为 '.0%' 只显示整数百分比
)

fig.update_layout(margin=dict(l=40, r=40, t=60, b=40))

fig.show()